<a href="https://colab.research.google.com/github/cmzwolf/JupyterVAMDCPortal/blob/main/SpeciesDB_dataExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VAMDC Species Database extractor


This notebook extracts all the species in the VAMDC species database (https://species.vamdc.org/) and save a timestamped standalone copy into an Excel file.

It also display the data locally in the notebook.

In [1]:
import pandas as pd
import json
import urllib.request
from datetime import datetime

In [2]:
# Getting the informations for the VAMDC nodes
urlNode = 'https://species.vamdc.org/web-service/api/v12.07/nodes'
responseNode = urllib.request.urlopen(urlNode)
json_list = responseNode.read()

data = json.loads(json_list)
df_nodes = pd.json_normalize(data)

In [4]:
from pickle import NONE

# Quering the Species Database VAMDC service
url = 'https://species.vamdc.org/web-service/api/v12.07/species'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# wrapping the results into a unique Pandas Dataframe (called AllSpeciesDF)

dataFrames = []
AllSpeciesDF = NONE
for key, value in data.items():
  datum = json.dumps(value)
  df = pd.read_json(datum, orient='records')
  df.insert(0,"ivoIdentifier", str(key))
  dataFrames.append(df)

AllSpeciesDF = pd.concat(dataFrames)
AllSpeciesDF = AllSpeciesDF.merge(df_nodes[['ivoIdentifier', 'shortName']], on='ivoIdentifier', how='left')


col = AllSpeciesDF.pop('shortName')  # Remove the column and store it in col
AllSpeciesDF.insert(0, 'shortName', col)  # Insert the column at the beginning

# Split the column 'lastSeenDateTime' and expand into two separate columns
AllSpeciesDF['lastIngestionScriptDate']= AllSpeciesDF["lastSeenDateTime"].apply(lambda x: x.split('||')[0])
AllSpeciesDF['speciesLastSeenOn']= AllSpeciesDF["lastSeenDateTime"].apply(lambda x: x.split('||')[1])

# Drop the original 'lastSeenDateTime' column
AllSpeciesDF = AllSpeciesDF.drop('lastSeenDateTime', axis=1)

# Call to_excel() function with the file name to export the DataFrame

now = datetime.now()
date_time = now.strftime("%Y%m%d_%H_%M")
file_name = "SpeciesDatabase"+date_time+".xlsx"

writer = pd.ExcelWriter(file_name, engine='openpyxl')

# convert the DataFrame to an Excel object
AllSpeciesDF.to_excel(writer, index=False, sheet_name='species')

# get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['species']

# iterate over the columns and set the width of each column based on the maximum length of the data in that column
for i, column in enumerate(AllSpeciesDF.columns):
    column_width = max(AllSpeciesDF[column].astype(str).map(len).max(), len(column))
    worksheet.column_dimensions[worksheet.cell(row=1, column=i+1).column_letter].width = column_width

#convert the Node DataFrame to an Excel object
df_nodes.to_excel(writer, index=False, sheet_name='nodes')
workbook = writer.book
worksheet = writer.sheets['nodes']

# iterate over the columns and set the width of each column based on the maximum length of the data in that column
for i, column in enumerate(df_nodes.columns):
    column_width = max(df_nodes[column].astype(str).map(len).max(), len(column))
    worksheet.column_dimensions[worksheet.cell(row=1, column=i+1).column_letter].width = column_width

# save the Excel file
import warnings
warnings.filterwarnings("ignore")
writer.save()

print("The excel file "+ file_name+" has been created in /content directory. You can download it using the file explorer on the left of this notebook")

The excel file SpeciesDatabase20231219_19_33.xlsx has been created in /content directory. You can download it using the file explorer on the left of this notebook


In [5]:
# display the species into a table in this notebook
AllSpeciesDF

,shortName,ivoIdentifier,InChI,InChIKey,stoichiometricFormula,massNumber,charge,speciesType,formula,name,did,lastIngestionScriptDate,speciesLastSeenOn
0,AMDIS Ionization,ivo://vamdc/amdis-ionization,InChI=1S/H/q-1,KLGZELKXQMTEMM-UHFFFAOYSA-N,H,1,-1,atom,None,Hydrogen negative ion -1,KLGZELKXQMTEMM-UHFFFAOYSA-N,2023-12-18,2023-12-18
1,AMDIS Ionization,ivo://vamdc/amdis-ionization,InChI=1S/H,YZCKVEUIGOORGS-UHFFFAOYSA-N,H,1,0,atom,None,Hydrogen,YZCKVEUIGOORGS-UHFFFAOYSA-N,2023-12-18,2023-12-18
2,AMDIS Ionization,ivo://vamdc/amdis-ionization,InChI=1S/p+1,GPRLSGONYQIRFK-UHFFFAOYSA-N,H,1,1,atom,None,Hydrogen positive ion 1,GPRLSGONYQIRFK-UHFFFAOYSA-N,2023-12-18,2023-12-18
3,AMDIS Ionization,ivo://vamdc/amdis-ionization,InChI=1S/He,SWQJXJOGLNCZEY-UHFFFAOYSA-N,He,4,0,atom,None,Helium,SWQJXJOGLNCZEY-UHFFFAOYSA-N,2023-12-18,2023-12-18
4,AMDIS Ionization,ivo://vamdc/amdis-ionization,InChI=1S/He/q+1,QLNXTEZOQCZJBA-UHFFFAOYSA-N,He,4,1,atom,None,Helium positive ion 1,QLNXTEZOQCZJBA-UHFFFAOYSA-N,2023-12-18,2023-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,Theoretical spectral database of polycyclic a...,ivo://vamdc/OA-Cagliari/PAH,1S/C28H14/c1-2-4-20-19(3-1)21-13-11-17-9-7-15-...,NQSLOOOUQZYGEB-UHFFFAOYSA-N,C28H14,350,0,molecule,$C_{28}H_{14}$,Benzo[a]coronene,NQSLOOOUQZYGEB-UHFFFAOYSA-N,2023-12-18,2023-12-18
4990,Theoretical spectral database of polycyclic a...,ivo://vamdc/OA-Cagliari/PAH,1S/C28H14/c1-5-15-13-16-6-3-11-21-22-12-4-8-18...,QDPTYXQGUBFJAB-UHFFFAOYSA-N,C28H14,350,0,molecule,$C_{28}H_{14}$,Bisanthene,QDPTYXQGUBFJAB-UHFFFAOYSA-N,2023-12-18,2023-12-18
4991,Theoretical spectral database of polycyclic a...,ivo://vamdc/OA-Cagliari/PAH,1S/C28H14/c1-5-15-13-16-6-3-11-21-22-12-4-8-18...,QOUBNBMMPKANCJ-UHFFFAOYSA-N,C28H14,350,0,molecule,$C_{28}H_{14}$,Bisanthene,QOUBNBMMPKANCJ-UHFFFAOYSA-N,2023-12-18,2023-12-18
4992,Theoretical spectral database of polycyclic a...,ivo://vamdc/OA-Cagliari/PAH,1S/C28H14/c1-5-15-13-16-6-3-11-21-22-12-4-8-18...,RGRVHYRLOKMHBA-UHFFFAOYSA-N,C28H14,350,0,molecule,$C_{28}H_{14}$,Bisanthene,RGRVHYRLOKMHBA-UHFFFAOYSA-N,2023-12-18,2023-12-18
